In [1]:
### Assumption : Confirmed_cases.csv exists in the working directory

from selenium import webdriver
import pandas as pd
import os
import re
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import random
from datetime import datetime
import collections

In [2]:
URL = "https://www.arcgis.com/apps/opsdashboard/index.html#/85320e2ea5424dfaaa75ae62e5c06e61"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = True
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
br = webdriver.Chrome(executable_path=r"C:\Users/qasim/Downloads/chromedriver_win32_v80/chromedriver.exe",options=chrome_options)
br.get(URL)
time.sleep(3)

In [4]:
data = br.find_element_by_id("ember21").text.split("\n")[:6]

In [5]:
def extract_num(string):
    string = string.split(" ")[0]
    return int(string.replace(",", ""))

In [6]:
logistics = {"CONFIRMED" : extract_num(data[1]), "DEATHS" : extract_num(data[3]), "RECOVERED" : extract_num(data[5])}

In [7]:
data = br.find_element_by_id("ember107").click()

In [8]:
TIME = str(datetime.now())
TIME = ':'.join(TIME.split(":")[:-1])

In [9]:
root = br.find_element_by_id("ember60")

In [10]:
br.find_element_by_id("ember66").screenshot("Images/"+ TIME.split(" ")[0] +".png")

True

In [11]:
data = root.text.split("\n")[1:100]

In [12]:
case = {}

In [13]:
for x in data:
    x = x.split(" ")
    country = " ".join(x[1:]).replace("*", "").strip()
    cases = int(x[0].replace(",",""))
    case[country] = cases

In [14]:
case = collections.OrderedDict(sorted(case.items()))

In [15]:
raw_data = {**logistics, **case}

In [16]:
df = pd.read_csv("Confirmed_case.csv")

In [17]:
df.head()

,Country,3/18/2020 17:44,17:44.1,3/18/2020 17:48,3/18/2020 18:19,3/18/2020 18:38
0,CONFIRMED,214894,214894,214894,214894,214894
1,DEATHS,8732,8732,8732,8732,8732
2,RECOVERED,83313,83313,83313,83313,83313
3,Afghanistan,22,22,22,22,22
4,Albania,59,59,59,59,59


In [18]:
conts = list(df["Country"])
curr_count = len(df["Country"])

In [19]:
zero_fill = np.zeros(df.shape[1] - 1)

In [20]:
for label in raw_data:
    if(label not in conts):
        temp = np.append(zero_fill, label)
        temp = temp[::-1]
        df.loc[curr_count] = temp
    

In [21]:
df.tail()

,Country,3/18/2020 17:44,17:44.1,3/18/2020 17:48,3/18/2020 18:19,3/18/2020 18:38
97,United Arab Emirates,113,113,113,113,113
98,United Kingdom,2642,2642,2642,2642,2642
99,Uruguay,50,50,50,50,50
100,Venezuela,36,36,36,36,36
101,Vietnam,75,75,75,75,75


In [22]:
df_temp = pd.DataFrame.from_dict(raw_data, orient="index", columns=[TIME]).reset_index()

In [23]:
del df_temp["index"]

In [24]:
df_cum = [df, df_temp]

In [25]:
df_cum = pd.concat(df_cum, axis=1)

In [26]:
df_cum.tail()

,Country,3/18/2020 17:44,17:44.1,3/18/2020 17:48,3/18/2020 18:19,3/18/2020 18:38,2020-03-18 18:40
97,United Arab Emirates,113,113,113,113,113,113
98,United Kingdom,2642,2642,2642,2642,2642,2642
99,Uruguay,50,50,50,50,50,50
100,Venezuela,36,36,36,36,36,36
101,Vietnam,75,75,75,75,75,75


In [27]:
null_values = df_cum[df_cum[TIME].isnull()].index.tolist()

In [28]:
if(len(null_values) > 0):
    priori = df_cum.columns[-2]
    df_copy = df_cum.copy()

    for x in null_values:
        value = df_copy[priori][x]
        df_cum.loc[:,TIME][x] = value

    del df_copy

In [29]:
df_cum.tail()

,Country,3/18/2020 17:44,17:44.1,3/18/2020 17:48,3/18/2020 18:19,3/18/2020 18:38,2020-03-18 18:40
97,United Arab Emirates,113,113,113,113,113,113
98,United Kingdom,2642,2642,2642,2642,2642,2642
99,Uruguay,50,50,50,50,50,50
100,Venezuela,36,36,36,36,36,36
101,Vietnam,75,75,75,75,75,75


In [30]:
df_cum.to_csv("Confirmed_case.csv", index=False)